In this notebook, we'll be going through the neural network implementation code from Chapter 1 in more detail. Along the way, a series of mini-exercises should help to explain some of the potentially confusing commands.

We'll be doing a bit of object-oriented Python in this study group - this means we'll be making different classes with their own methods (~functions), which can be instantiated to make objects of that class. See the email and repo for a quick review of some key ideas. 

In [193]:
import numpy as np
import random

# Let's start by making the Network class
class Network(object):

    # The __init__ method is a sort of Python equivalent of a constructor
    # Constructors initialize the values of variables of a newly constructed object
    # When we make a new network object, we'll pass a certain value of 'sizes' as an arguments
    # For example, n1 = Network([2,5,1]) makes a new Network object with the sizes variable set to [2,5,1] which means
    # we'll have 2 input neurons, 5 in hidden layer, 1 in output
    # So, we can make networks with different values of sizes which initialise the network differently and the
    # sizes argument specifies the number of neurons in each layer of the network
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        # we randomly initialise the biases of the hidden layer and output neurons only (hence sizes[1:])
        # the numpy random.randn command generates numbers from a standard normal distribution (mean 0, variance 1)
        # This is a use of a list comprehension, which are a powerful tool in python
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        # here, we initialise the weights leading from the input to the hidden layer, and 
        # from the hidden layer to the output. Play around with the indexing and zip command to get a feel for it
        self.weights = [np.random.randn(y, x) 
                        for x, y in zip(sizes[:-1], sizes[1:])]

In [217]:
# Some useful commands you could try out to help understand what is happening in the above code
# Click "insert cell below" to get a new box you can type the commands into
# Typing commands yourself > copying/pasting commands or typing print in front of commands > reading

# randn command
np.random.randn(1,1)
np.random.randn(5,1)
np.random.randn(5,3)

# array indexing
sizes = [2,5,1]
sizes[1:]
sizes[:-1]
zip(sizes[:-1], sizes[1:])

# list comprehensions
# reminder of some simpler uses
[x for x in range(10)]
[x**2 for x in range(10)]
[x**2 for x in range(10) if x%2==0]

# leading up to understanding the specific usage in the Network class definition
[x+y for x, y in zip(sizes[:-1], sizes[1:])]
[x*y for x, y in zip(sizes[:-1], sizes[1:])]
[x for x, y in zip(sizes[:-1], sizes[1:])]
[[x,y] for x, y in zip(sizes[:-1], sizes[1:])]
[np.random.randn(x) for x, y in zip(sizes[:-1], sizes[1:])]
[np.random.randn(x, y) for x, y in zip(sizes[:-1], sizes[1:])]
[np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]

[array([[-1.08351413, -1.44065192],
        [ 0.32184529, -0.64573423],
        [-1.03064089,  0.54424213],
        [ 0.06212166,  1.41520053],
        [-0.15007144,  0.98580898]]),
 array([[-0.13527947,  1.18161082,  0.5471675 , -0.53280803,  0.04046108]])]

In [36]:
# Let's go ahead and make a new network
net1 = Network([2,5,1])
print net1.num_layers
print net1.biases
print net1.weights

3
[array([[ 0.1265344 ],
       [ 0.92402966],
       [-1.62969926],
       [ 1.62903212],
       [ 0.41219127]]), array([[ 0.83005241]])]
[array([[-0.59216698,  0.57756099],
       [-1.05164244,  1.77603307],
       [ 0.7664931 ,  0.05062822],
       [-1.40892278,  0.44292187],
       [-2.44369652,  1.38992025]]), array([[-1.75631984, -1.43950517, -0.39580496, -1.01950956, -0.03196487]])]


In [37]:
# Let's make the sigmoid function
# Refer to the textbook for an explanation of the function's shape (an S, a sort of smoothed step function)
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

In [325]:
# Check sigmoid function works as intended
sigmoid(0)
sigmoid(1)
sigmoid(-1)
sigmoid(10)
sigmoid(-10)

4.5397868702434395e-05

In [226]:
# Next up, let's make the feedforward method of the Network class 
# we'll put this code into the correct place later, let's understand it first
# We are trying to apply the sigmoid function to the input from each layer
# I.e. each layer applies the sigmoid function on the layer before it
# I got rid of the references to 'self' for now, for clarity
def feedforward(a):
    for b, w in zip(biases, weights):
        print "The weights we're working with are:"; print w
        print "The biases we're working with are:"; print b
        a = sigmoid(np.dot(w, a)+b)
        print "We put the weighted input + bias into the sigmoid function, which for this layer gives activations of:",; print a; print "done printing a"
    return a

In [116]:
# So we can see what's happening, let's play with numbers
biases = net1.biases
weights = net1.weights
# recall these are the biases of the 5 hidden layers and the 1 output layer
biases
# these are the weights leading from each of the 2 neurons in the input to each of the 5 neurons in the hidden layer
# and the weights from the 5 input neurons to the 1 output neuron
weights
# Recall that for each e.g. hidden layer neuron, we need to multiply all incoming input by weights and sum it up and add a bias
# Then we put this value through the sigmoid function
# So, we need an efficient way of multiplying a lot of weights and a lot of inputs (activations) together
# We'll use the dot product for this, np.dot(weights, inputs)
# let's see what np.dot does
np.dot([1,1,1], [5,0,1])
np.dot([3,1,2], [0,0,0])
# looks good to me! We multiply element wise and add these values up
# we can immediately add a bias to the output
np.dot([3,3,3], [1,1,1]) + 1
# then we'll want to apply the sigmoid function to get the activation of the next layer of neurons
z = np.dot([3,3,3], [1,1,1]) + 1
sigmoid(z)
# that's quite a high input to the sigmoid function, let's try something more realistic
z = np.dot([-0.3,0.2,0.5], [0.4,0.2,-0.1]) + 0.5
sigmoid(z)

0.59145897843278006

In [234]:
# what on earth does zip(biases, weights) look like? 
# if you're also finding it hard to visualise all these different matrices, try:
import pandas as pd
pd.DataFrame([1,2])
pd.DataFrame([3,4])
pd.DataFrame(zip([1,2],[3,4]))
# now we can look at these more clearly:
pd.DataFrame(biases)
pd.DataFrame(weights)
pd.DataFrame(zip(biases,weights))
# finally, try to understand:
biases
weights
zip(biases, weights)

[(array([[ 0.1265344 ],
         [ 0.92402966],
         [-1.62969926],
         [ 1.62903212],
         [ 0.41219127]]), array([[-0.59216698,  0.57756099],
         [-1.05164244,  1.77603307],
         [ 0.7664931 ,  0.05062822],
         [-1.40892278,  0.44292187],
         [-2.44369652,  1.38992025]])),
 (array([[ 0.83005241]]),
  array([[-1.75631984, -1.43950517, -0.39580496, -1.01950956, -0.03196487]]))]

In [ ]:
# Hope that helps! Let's move on to the mini-batch stochastic gradient descent function
# We will have to take training data and the desired outputs and move the weights and bias vectors towards values 
# that get close to computing the desired output
# I am ignoring the test_data option and the self references 
def SGD(training_data, epochs, mini_batch_size, eta, test_data=None):

    n = len(training_data)
    
    # the xrange function is very similar to range(generates integers from 0 to whatever-1) but doesn't generate a static list at run time
    # it only generates integers when they're needed, this is good when lists are large and the system is memory sensitive
    for j in xrange(epochs):
        
        # random.shuffle shuffles the training data in place
        random.shuffle(training_data)
        
        # let's create our mini batches of size mini_batch_size
        mini_batches = [
            training_data[k:k+mini_batch_size]
            for k in xrange(0, n, mini_batch_size)]
        
        # then, for each mini_batch, we "update" (i.e. compute gradients and update weights and biases)
        for mini_batch in mini_batches:
            self.update_mini_batch(mini_batch, eta)
        
        # an epoch is complete when all mini-batches have been processed
        print "Epoch {0} complete".format(j)

In [147]:
# Decomposing the SGD function
# first, let's try out the random.shuffle function
# notice the function doesn't return anything, but if you check back on x after shuffling, it's been shuffled
x = [1,2,3,4]
random.shuffle(x)
x
# now try in 2 dimensions like with our training input
x = [[1,2], [3,4], [5,6]]
random.shuffle(x)
x

[[5, 6], [1, 2], [3, 4]]

In [326]:
# Next up, let's get to grips with how the mini batches are created
# let's make up some specific values for variables so we can follow them through
training_data = [[1,1], [1,2], [2,3], [3,4], [5,7], [7,8]]
n = len(training_data)
mini_batch_size = 2

# recall that range and xrange work by specifying the starting value, the end value, and the step size
# so xrange(0, n, mini_batch_size), in our case xrange(0,6,2), will produce the output [0,2,4]
xrange(0, n, mini_batch_size)
range(0, n, mini_batch_size)

# this produces start indices which we can use to split our shuffled training data set on
training_data[0:0+mini_batch_size]
training_data[2:2+mini_batch_size]

# all together, let's now run:
# this divides up our training data set of 6 data points into mini-batches of 2 data points each
mini_batches = [training_data[k:k+mini_batch_size] for k in xrange(0, n, mini_batch_size)]
mini_batches

# note that we can break up list comprehensions onto two lines to improve readability
mini_batches = [training_data[k:k+mini_batch_size] 
                for k in xrange(0, n, mini_batch_size)]


In [327]:
# quick review of python print formatting
# just printing a string and specifying the separator
print "Epoch {0} complete".format(1)
print '{0} and {1}'.format('Jake', 'Finn')
print '{1} and {0}'.format('Jake', 'Finn')

# play with numbers
print "Come on grab your {0} friends and {1} princesses".format(47,11.9557) 
# print the first value (index of 0) to fill 5 spaces (d means integer)
print "Come on grab your {0:5d} friends and {1} princesses".format(47,11.9557) 

# print the first value (index of 0) to fill 5 spaces, write it as a float, with precision of 2 decimal places
print "Come on grab your {0:5.2f} friends and {1} princesses".format(47,11.9557) 

# write the sentence sensibly. These two statements are equivalent
print "Come on grab your {0:1d} friends and {1:4.2f} princesses".format(47,11.9557)
print "Come on grab your {0} friends and {1:.2f} princesses".format(47,11.9557)

Epoch 1 complete
Jake and Finn
Finn and Jake
Come on grab your 47 friends and 11.9557 princesses
Come on grab your    47 friends and 11.9557 princesses
Come on grab your 47.00 friends and 11.9557 princesses
Come on grab your 47 friends and 11.96 princesses
Come on grab your 47 friends and 11.96 princesses


In [307]:
# Great! For each step in the training phase (each 'epoch'), you'll have noticed that we updated each mini_batch with the
# update_mini_batch method. Now let's take a look at what this is 
# Intuitively, we want to gradually move the weights and biases towards the values that produce the correct output from the whole network
# The rate at which we move towards this optimum depends on the learning rate, eta

def update_mini_batch(mini_batch, eta):
    # 'nabla' is the word for the upside down triangle symbol we've been using to denote gradient vectors
    # np.zeros produces zeros in the same shape as the biases and weights
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    for x, y in mini_batch:
        
        # we use backpropagation to get gradient vectors, treating as black box at the moment
        delta_nabla_b, delta_nabla_w = backprop(x, y)
        # we update the gradient vectors
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
    
    # now, we're updating the weights and bias vectors by the opposite of the gradients 
    # this updates the parameters to values that decrease the cost function
    weights = [w-(eta/len(mini_batch))*nw 
                    for w, nw in zip(weights, nabla_w)]
    biases = [b-(eta/len(mini_batch))*nb 
                    for b, nb in zip(biases, nabla_b)]

In [184]:
# try out the zeros function
np.zeros(5)
np.zeros([2,5])
# try out the shape function
x = np.array([1,2,3])
x.shape
x = np.array([[1,2],[4,5]])
x.shape
x = np.array([[1,2,3],[4,5,6]])
x
x.shape
# trying it out on our data set
weights
weights[0].shape

(5, 2)

In [324]:
# Trying out the update mini batch function
# the gradient vectors have the same shape as the bias and weights vectors
# because every parameter (bias or weight) contributes to the cost function at some rate, i.e. has a dC/dp
# I'll just set some parameters equal to arbitrary values so you can track how weights and biases are updated as 
# a function of the gradient vectors
nabla_b = [np.zeros(b.shape) for b in biases]
nabla_w = [np.zeros(w.shape) for w in weights]
#print nabla_b

eta = 0.05

# next up, backprop returns a tuple that tells us how to update the weights and biases.
# let's make up some values first
for mini_batch in mini_batches:
    
    for x, y in mini_batch:
        
        print x,y

        delta_nabla_b = [np.random.rand(x,y)*0.01 for x,y in (b.shape for b in biases)]
        delta_nabla_w = [np.random.rand(x,y)*0.01 for x,y in (w.shape for w in weights)]
        
        # let's print out what the random delta nablas are
        print delta_nabla_b
        print delta_nabla_w

        # we update the gradient vectors
        nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

# updates the parameters 
weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(weights, nabla_w)]
biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(biases, nabla_b)] 

print weights
print biases

1 1
[array([[ 0.00828323],
       [ 0.00608295],
       [ 0.00717254],
       [ 0.00888058],
       [ 0.0058908 ]]), array([[ 0.00812181]])]
[array([[ 0.00140011,  0.00135424],
       [ 0.00088149,  0.00652609],
       [ 0.00344452,  0.00186034],
       [ 0.00998321,  0.00495353],
       [ 0.00735936,  0.0024171 ]]), array([[  6.66871643e-03,   3.91447667e-04,   3.51334990e-03,
          4.17377002e-05,   2.14327866e-03]])]
1 2
[array([[ 0.00349216],
       [ 0.00927524],
       [ 0.00730807],
       [ 0.00933622],
       [ 0.00714726]]), array([[ 0.00473516]])]
[array([[ 0.00474388,  0.00111355],
       [ 0.00941205,  0.0032536 ],
       [ 0.00799527,  0.00621469],
       [ 0.00927401,  0.00823712],
       [ 0.00846398,  0.00529215]]), array([[ 0.00758654,  0.0044922 ,  0.00248039,  0.00498939,  0.00518081]])]
2 3
[array([[ 0.00214758],
       [ 0.00520195],
       [ 0.00779013],
       [ 0.0064365 ],
       [ 0.0025817 ]]), array([[ 0.00923977]])]
[array([[ 0.00615336,  0.00317071],


In [291]:
# Let's have a peak at backprop method, which will be covered a fair amount in Chapter 2 
# this method returns (nabla_b, nabla_w) which is the gradient of the cost function
# specifically, they are the partial derivatives of the cost function for each given parameter (each weight, each bias term)
# we need to know these gradients for each parameter to figure out which direction to push them in
def backprop(x, y):
    
    # we begin by knowing nothing about the partial derivatives - we initialise the gradient vectors (nablas) with zeros
    nabla_b = [np.zeros(b.shape) for b in biases]
    nabla_w = [np.zeros(w.shape) for w in weights]
    
    activation = x
    activations = [x] # list to store all the activations, layer by layer
    zs = [] # list to store all the z vectors, layer by layer
    
    # given some activation, we multiply by the weights and add the biases to get the activity at the next layer over
    for b, w in zip(biases, weights):
        z = np.dot(w, activation)+b
        zs.append(z)
        activation = sigmoid(z)
        activations.append(activation)
        
        # we haven't covered this yet, but in back propagation the error is 'passed backwards' through the network
        delta = cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
       
        # l = 1 means the last layer of neurons, l = 2 is the second-last layer, and so on.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)